In [1]:
import secret_keys as key
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import pandas as pd

C:\Users\i323570\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
import tweepy

# Creating the authentication object
auth = tweepy.OAuthHandler(key.consumer_key, key.consumer_secret)
# Setting your access token and secret
auth.set_access_token(key.access_token, key.access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth,wait_on_rate_limit=True) 

In [3]:
# Search for Tweets tweeted from in and around Bangalore
latitude = 12.971599    # geographical centre of search
longitude = 77.594563    # geographical centre of search
max_range = 1000           # search range in kilometres
num_results = 1000      # minimum results to obtain

search_words = '#corona OR #covid19 OR #covid OR #vaccine OR #bangalore'
date_since = '2020-04-10'

In [4]:
tweets = tweepy.Cursor(api.search ,
                       q =search_words,
                       lang = 'en',
                       geocode = "%f,%f,%dkm" % (latitude, longitude, max_range),
                       since = date_since
                      ).items(num_results)

In [5]:
tweet_list = [[ tweet.geo,tweet.user.geo_enabled, tweet.text, tweet.user.screen_name, tweet.user.location ] \
              for tweet in tqdm_notebook(tweets,total = num_results) 
              if (not tweet.retweeted) and ('RT @' not in tweet.text)]

In [6]:
df_tweets = pd.DataFrame(tweet_list, columns = ['geo','geo_enabled', 'tweet' , 'user_handle','location'])

In [7]:
df_tweets.tail()

,geo,geo_enabled,tweet,user_handle,location
201,None,True,Lockdown is in no way a solution to the #COVID...,dnyadav,"Uttar Pradesh, India"
202,None,True,"As #IndiaFightsCorona, take a look at this com...",AkashvaniAIR,India
203,None,True,Biggest weapon against #COVID19 is testing-tes...,ANI,India
204,None,True,#Karnataka - 34 more #COVID19 cases reported i...,rajeshkshitij,hyderabad
205,None,True,"It’s The Math, Stupid.\n\n#Corona #Covid_19 \n...",MeBhavya,Made In India


In [8]:
df_tweets.to_csv('../Data/tweets_with_location_raw_16th_April.csv',index=False)

In [9]:
# https://towardsdatascience.com/another-twitter-sentiment-analysis-bb5b01ebad90
# Tweet data cleaning
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
#     lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
#     words = tok.tokenize(lower_case)
    words = tok.tokenize(letters_only)
    return (" ".join(words)).strip()

In [10]:
df_tweets['cleaned_tweets'] = df_tweets.tweet.progress_apply(tweet_cleaner)

In [11]:
df_tweets.head()

,geo,geo_enabled,tweet,user_handle,location,cleaned_tweets
0,None,True,ବିନାଶ କାଳେ ବିପରୀତ ବୁଦ୍ଧି ।\nPeople order as mu...,SanjitaPanigra2,"Bhubaneshwar, India",People order as much as pizza on line food you...
1,None,False,"In the harsh times of this #Pandemic, Atom urg...",AtomTech_India,"Mumbai, Maharashtra",In the harsh times of this Pandemic Atom urges...
2,None,True,"@barandbench @MoHFW_INDIA Next, a petition dem...",kochattil,Hyderabad,INDIA Next a petition demanding that Covid pat...
3,None,True,9 more #COVID19 cases reported in Haryana toda...,ANI,India,more COVID cases reported in Haryana today in ...
4,None,True,Self-discipline and self-motivation is the key...,ecosmob,India,Self discipline and self motivation is the key...


In [12]:
df_tweets['cleaned_tweets'] = df_tweets.cleaned_tweets.str.replace('RT', '')

In [13]:
df_tweets.head()

,geo,geo_enabled,tweet,user_handle,location,cleaned_tweets
0,None,True,ବିନାଶ କାଳେ ବିପରୀତ ବୁଦ୍ଧି ।\nPeople order as mu...,SanjitaPanigra2,"Bhubaneshwar, India",People order as much as pizza on line food you...
1,None,False,"In the harsh times of this #Pandemic, Atom urg...",AtomTech_India,"Mumbai, Maharashtra",In the harsh times of this Pandemic Atom urges...
2,None,True,"@barandbench @MoHFW_INDIA Next, a petition dem...",kochattil,Hyderabad,INDIA Next a petition demanding that Covid pat...
3,None,True,9 more #COVID19 cases reported in Haryana toda...,ANI,India,more COVID cases reported in Haryana today in ...
4,None,True,Self-discipline and self-motivation is the key...,ecosmob,India,Self discipline and self motivation is the key...


In [14]:
df_tweets.to_csv('../Data/Tweets_scraped_2000_16th_April_CORONA_COVID.csv',index=False)

In [137]:
# import tweepy
# import json

# class StreamListener(tweepy.StreamListener):
#     def on_status(self, status):
#         print(status.id_str)
#         # if "retweeted_status" attribute exists, flag this tweet as a retweet.
#         is_retweet = hasattr(status, "retweeted_status")

#         # check if text has been truncated
#         if hasattr(status,"extended_tweet"):
#             text = status.extended_tweet["full_text"]
#         else:
#             text = status.text

#         # check if this is a quote tweet.
#         is_quote = hasattr(status, "quoted_status")
#         quoted_text = ""
#         if is_quote:
#             # check if quoted tweet's text has been truncated before recording it
#             if hasattr(status.quoted_status,"extended_tweet"):
#                 quoted_text = status.quoted_status.extended_tweet["full_text"]
#             else:
#                 quoted_text = status.quoted_status.text

#         # remove characters that might cause problems with csv encoding
#         remove_characters = [",","\n"]
#         for c in remove_characters:
#             text.replace(c," ")
#             quoted_text.replace(c, " ")

#         with open("out.csv", "a", encoding='utf-8') as f:
#             f.write("%s,%s,%s,%s,%s,%s\n" % (status.created_at,status.user.screen_name,is_retweet,is_quote,text,quoted_text))

#     def on_error(self, status_code):
#         print("Encountered streaming error (", status_code, ")")
#         sys.exit()


# if __name__ == "__main__":

#     # These are provided to you through the Twitter API after you create a account
#     consumer_key = key.consumer_key
#     consumer_secret = key.consumer_secret
#     access_token = key.access_token
#     access_token_secret = key.access_token_secret
    
#     auth1 = tweepy.OAuthHandler(consumer_key, consumer_secret)
#     auth1.set_access_token(access_token, access_token_secret)

#     # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the 
#     # geographic area from which you will stream tweets. The first two define the southwest
#     # corner of the box and the second two define the northeast corner of the box. 
#     LOCATIONS = [77.352731,12.803097,77.800337,13.092186]        # Bangalore

#     stream_listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True))
#     stream = tweepy.Stream(auth=auth1, listener=stream_listener)
#     stream.filter(locations=LOCATIONS)

1245289633497272320
1245289663020974081
1245289669190758406
1245289696718024709
1245289705031098368
1245289705798680577
1245289755895529472
1245289775663206401
1245289834517651457
1245289857389187078
1245289953321312258
1245290001790717952
1245290002159820802
1245290025358524417
1245290056425697281


KeyboardInterrupt: 